In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 10

#path for Kaggle dataset
BASE_DIR = '/kaggle/input/cub2002011/CUB_200_2011/images'

selected_classes = [
    '001.Black_footed_Albatross',
    '014.Indigo_Bunting',
    '050.Eared_Grebe',
    '061.Horned_Grebe',
    '073.Blue_Jay',
    '095.Baltimore_Oriole',
    '116.Painted_Bunting',
    '144.Pacific_Loon',
    '172.Green_tailed_Towhee',
    '199.Western_Meadowlark'
]

def create_bird_cnn_model():
    model = models.Sequential([
        #1st convolutional block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),

        #2nd convolutional block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        #3rd convolutional block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        #4th convolutional block
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        #flatten and dense layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')  # Output layer for 25 classes
    ])
    return model

#graphs
def plot_training_curves(history):
    plt.figure(figsize=(12, 4))

    #accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    #loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

def main():
    #data augmentation and preprocessing
    datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        validation_split=0.2,
        horizontal_flip=True,
        rotation_range=30,
        zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2
    )

    #training data generator
    train_generator = datagen.flow_from_directory(
        directory=BASE_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        classes=selected_classes
    )

    #validation data generator
    val_generator = datagen.flow_from_directory(
        directory=BASE_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        classes=selected_classes  # Only use the selected 25 species
    )

    #create model
    model = create_bird_cnn_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    #train model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=35,
        verbose=1
    )

    #plot training curves
    plot_training_curves(history)

    #save model
    model.save('bird_species_classifier.h5')
    print("Model saved as bird_species_classifier.h5")

if __name__ == "__main__":
    main()

Found 240 images belonging to 10 classes.
Found 60 images belonging to 10 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │    18,874,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,268,426 (73.50 MB)

 Trainable params: 19,268,426 (73.50 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.1613 - loss: 2.3446 - val_accuracy: 0.3667 - val_loss: 1.7133
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 158ms/step - accuracy: 0.2229 - loss: 1.7779 - val_accuracy: 0.2000 - val_loss: 1.5353
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 0.3023 - loss: 1.6027 - val_accuracy: 0.4000 - val_loss: 1.2529
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 162ms/step - accuracy: 0.4065 - loss: 1.3913 - val_accuracy: 0.5167 - val_loss: 1.1934
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.5038 - loss: 1.2395 - val_accuracy: 0.5167 - val_loss: 0.9192
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 162ms/step - accuracy: 0.4875 - loss: 1.2152 - val_accuracy: 0.4500 - val_loss: 1.0404
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 161ms/step - accuracy: 0.4707 - loss: 1.1337 - val_accuracy: 0.5000 - val_loss: 0.9560
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.3983 - loss: 1.1253 - val_accuracy: 0.5500 - val_loss: 0

KeyboardInterrupt: 